#Task 1: Import Libraries

In [ ]:
!pip install transformers torch accelerate

In [ ]:
import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

#Task 2: Load Lightweight model


In [ ]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

print("Model Loaded Successfully ✅")

#Task 3: Initialise FAQ knowledge Base

In [ ]:
faq_dict = {
    "return policy": "Our return policy allows returns within 30 days with a valid receipt.",
    "refund": "Refunds are processed within 5-7 business days after approval.",
    "shipping": "Standard shipping takes 3-5 business days.",
    "cancel order": "To cancel your order, please provide your order ID.",
    "contact": "You can contact our support team at support@company.com."
}


#Task 4: FAQ Detection Function


In [ ]:
def check_faq(user_input):
    user_input = user_input.lower()
    for key in faq_dict:
        if key in user_input:
            return faq_dict[key]
    return None

In [ ]:
#Conversation memory
conversation = [
    {"role": "system", "content": "You are a professional and polite customer service assistant. Always reply clearly and briefly."}
]

#Task 5: AI Response Generator(With Memory)

In [ ]:
def generate_ai_response(user_input):

    global conversation

    conversation.append({"role": "user", "content": user_input})

    # Apply chat template
    prompt = tokenizer.apply_chat_template(
        conversation,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=80,
        temperature=0.7,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id
    )

    reply = tokenizer.decode(outputs[0], skip_special_tokens=True)

    reply = reply.split("assistant")[-1].strip()

    conversation.append({"role": "assistant", "content": reply})

    return reply

#Task 6: Initialise Chatbot Conversation

In [ ]:
print("======================================")
print("🤖 Welcome to Smart Customer Service Chatbot")
print("Type 'exit' to end conversation")
print("======================================\n")

#Task 7: Testing section



In [ ]:
test_questions = [
    "What is your return policy?",
    "I want a refund",
    "My order is delayed"
]

print("Running Test Cases...\n")

for question in test_questions:
    print("You:", question)

    faq_response = check_faq(question)

    if faq_response:
        print("Chatbot (FAQ):", faq_response)
    else:
        print("Chatbot (AI):", generate_ai_response(question))

    print("-" * 50)

#Interactive Chat

In [ ]:
while True:
    user_input = input("You: ")

    if user_input.lower() == "exit":
        print("Chatbot: Thank you for contacting us. Have a great day! 😊")
        break

    faq_response = check_faq(user_input)

    if faq_response:
        print("Chatbot (FAQ):", faq_response)
    else:
        reply = generate_ai_response(user_input)
        print("Chatbot (AI):", reply)